In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm

In [71]:
class ThompsonSampleGauss:
    """
    A class to perform Gaussian Thompson Sampling on the multi-armed bandit problem.
    """
    
    def __init__(self, bandits: int, means: np.ndarray, stds: np.ndarray, pulls: int) -> None:
        """
        Initializes the ThompsonSampleTest class.

        Args:
            bandits: the number of arms in the bandit problem.
            means: a numpy array representing the true means of the bandits.
            stds: a numpy array representing the true standard deviations of the bandits.
            pulls: the number of times to pull the arms of the bandit problem.
        """
        self.bandits = bandits
        self.prior_means = np.zeros(bandits)
        self.prior_stds = np.ones(bandits) * 1000
        self.actual_means = means
        self.actual_stds = stds
        self.pulls = pulls
        self.samples = np.zeros(bandits)
        self.rewards = np.zeros(bandits)
        self.mean_sequences = {n : [] for n in range(bandits)}
        self.stds_sequences = {n : [] for n in range(bandits)}
        
        for n in range(bandits):
            self.mean_sequences[n].append(0)
        for n in range(bandits):
            self.stds_sequences[n].append(1)
        
    def get_choice(self) -> int:
        """
        Samples the prior distributions to select the arm to pull. 
        Note that we sample it twice to get both a mean and a standard deviation.        
        
        Returns:
            The index of the arm to pull.
        """       
        sample_means = norm.rvs(self.prior_means, self.prior_stds)
        sample_stds = np.abs(norm.rvs(self.prior_means, self.prior_stds))
        
        return np.argmax(norm.rvs(sample_means, sample_stds))
    
    def get_reward(self, choice: int) -> float:        
        """
        Samples the distribution of the arm chosen to get a reward.

        Args:
            choice: The index of the arm chosen.
            
        Returns:
            The reward obtained from the chosen arm.
        """        
        reward = norm.rvs(self.actual_means[choice], self.actual_stds[choice])
        
        return reward
    
    def update_prior(self, choice: int, reward: float) -> None:
        """
        Updates the prior distribution for the chosen arm.

        Args:
            choice: The index of the arm chosen.
            reward: The reward obtained from the chosen arm.
        """       
        self.samples[choice] += 1
        self.prior_means[choice] = (self.prior_means[choice] * (self.samples[choice] - 1) + reward) / self.samples[choice]
        self.prior_stds[choice] = np.sqrt((self.prior_stds[choice]**2 * (self.samples[choice] - 1) + (reward - self.prior_means[choice])**2) / self.samples[choice])
        
        self.mean_sequences[choice].append(self.prior_means[choice])
        self.stds_sequences[choice].append(self.prior_stds[choice])
                                          
    def do_it(self) -> None:
        """
        Performs the Thompson Sampling algorithm for the specified number of pulls.
        """        
        for i in range(self.pulls):

            choice = self.get_choice()
            reward = self.get_reward(choice)
            
            self.rewards[choice] += reward            
            self.update_prior(choice, reward)
    
    def compute_regret(self) -> None:
        """
        Calculates the fraction of the expected total rewards if the best
        distribution was chosen every time.
        
        Returns: the fraction of maximum expected returns
        """
        
        return sum(self.rewards) / (self.pulls * max(self.actual_means))

In [76]:
# generating random sets of means and standard deviations
means = norm.rvs(100, 125, size=100)
stds = np.abs(norm.rvs(52, 25, size=100))

# instantiating the class
test = ThompsonSampleGauss(len(means), means, stds, 500)

# running the thompson sampling algorithm
test.do_it()

# converged?
print("Found best:", np.argmax(means) == np.argmax(test.prior_means))

# total regret
print("Fraction of MaxEV achieved:", round(test.compute_regret(), 4))

Found best: True
Fraction of MaxEV achieved: 0.6825


In [77]:
# lets see a dataframe 
df = pd.DataFrame({"final_est_mean": test.prior_means,
                   "actual_mean" : means,
                   "final_est_std" : test.prior_stds,
                   "actual_std" : stds,
                   "sample_count" : test.samples})

df.sort_values(by='final_est_mean', ascending=False).head(15)

,final_est_mean,actual_mean,final_est_std,actual_std,sample_count
32,342.295047,351.068871,67.547560,62.645466,42.0
94,321.902949,320.899454,33.630141,39.620696,45.0
43,308.051171,304.299036,64.848582,59.759138,44.0
70,287.569764,284.677633,38.181354,46.028715,23.0
37,282.407403,283.997965,41.693814,65.595481,17.0
67,276.204791,291.196007,102.042351,102.905534,35.0
60,272.717090,273.144848,24.209628,24.793544,22.0
65,265.253324,269.829866,67.230606,57.728181,34.0
5,262.886127,244.095745,38.701340,60.296734,16.0
89,253.071412,226.261695,27.612864,46.356121,9.0
